In [3]:
# filename: Data Exploration
# author: Marx Wang

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
# import statsmodels.api as sm
# import scipy

In [5]:
df = pd.read_csv("data/alldata.csv")

In [12]:
df.head()

Index(['subjectkey', 'aggressive_sumscore', 'prosocial_child',
       'prosocial_parent', 'asr_scr_perstr_t', 'asr_scr_anxdep_t',
       'asr_scr_withdrawn_t', 'asr_scr_somatic_t', 'asr_scr_thought_t',
       'asr_scr_attention_t', 'asr_scr_aggressive_t', 'asr_scr_rulebreak_t',
       'asr_scr_intrusive_t', 'asr_scr_internal_t', 'asr_scr_external_t',
       'asr_scr_totprob_t', 'asr_scr_depress_t', 'asr_scr_anxdisord_t',
       'asr_scr_somaticpr_t', 'asr_scr_avoidant_t', 'asr_scr_adhd_t',
       'asr_scr_antisocial_t', 'asr_scr_inattention_t',
       'asr_scr_hyperactive_t', 'crpbi_bothcare', 'parent_monitor_y',
       'kbi_p_conflict', 'kbi_p_c_best_friend', 'kbi_p_c_reg_friend_group',
       'kbi_p_c_bully', 'kbi_p_c_mh_sa', 'fes_youth', 'fes_p_ss_fc_pr',
       'macv_p_ss_fs', 'macv_p_ss_fo', 'macv_p_ss_isr', 'macv_p_ss_fr',
       'macv_p_ss_r', 'demo_prnt_age_v2', 'demo_prnt_marital_v2',
       'demo_comb_income_v2', 'demo_fam_exp', 'demo_yrs_1', 'demo_yrs_2',
       'parent_rule

In [11]:
ag = pd.read_csv("data/aggression.csv")
ag.head()

,subjectkey,aggressive_sumscore,conduct,cbcl_scr_syn_rulebreak_t,cbcl_scr_syn_aggressive_t,cbcl_scr_dsm5_conduct_t,zconduct,zrule,zaggression,zconduct_dsm,Unnamed: 10,Unnamed: 11
0,NDAR_INV003RTV85,-1.613056,0,52,50,50,-0.394586,-0.158614,-0.512484,-0.547372,NaN,-1.613056
1,NDAR_INV005V6D2C,-2.020651,0,50,50,50,-0.394586,-0.566209,-0.512484,-0.547372,NaN,NaN
2,NDAR_INV007W6H7B,-2.020651,0,50,50,50,-0.394586,-0.566209,-0.512484,-0.547372,NaN,NaN
3,NDAR_INV00BD7VDC,-1.840037,0,50,51,50,-0.394586,-0.566209,-0.331870,-0.547372,NaN,NaN
4,NDAR_INV00CY2MDM,9.133184,5,57,68,69,2.647005,0.860373,2.738568,2.887238,NaN,NaN
